In [ ]:
import numpy as np
import pandas as pd
import math

# Total budget for the year
TotalBudget = 1000000000;

# Mapping of genre to number of movies of that genre to be produced this year
GenreWeights = {
    'Action' : -9.135e+06,
    'Family' : -2.153e+07,
    'Drama' : -1.128e+07,
    'Comedy': -2.876e+06,
    'Horror' : 6.371e+06,
    'Music' : -1.966e+07,
    'Science Fiction' : -4.979e+06,
    'Animation' : 8.404e+07,
    'Family' : -2.153e+07,
    'History': -8.120e+07
}

MovieGenres = ['Comedy', 'Action', 'Action', 'Drama', 'Family', 'Horror', 'Animation', 'Science Fiction', 'Music', 'History']

NumMovies = len(MovieGenres)

Actors = ['Maggie Smith', 'Tom Hanks', 'Whoopi Goldberg', 'Orlando Bloom', 'Willem Dafoe', 'Bruce Willis', 'Alan Rickman', 'Scarlett Johansson', 'Leonardo DiCaprio', 'Chris Evans']
ActorWeights = np.array([1.509e+07, 4.717e+07, 7.248e+07, -9.554e+07, 8.829e+06, 2.658e+06, -7.853e+06, -1.621e+07 , -5.958e+07, -8.787e+06])

NumActors = 10;

MinRuntime = 60;
MaxRuntime = 240;

MinBudget = 30000000;
MaxBudget = 250000000;

ReleaseYear = 2023;


In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

<gurobipy.Env, Parameter changes: OutputFlag=0>

In [ ]:
# Initialize the model
m = gp.Model(env=env)

### Decision Variables ###

# A vector of binary decision variables that denote which actors are selected for which movies
ActorDec = m.addMVar((NumMovies, NumActors), vtype=GRB.BINARY)

# A vector of decision variables that denote how much budget each movie gets
Budget = m.addMVar((NumMovies, 1))

# A vector of decision variables that denote how long each movie is
Runtime = m.addMVar((NumMovies, 1))


In [ ]:
### Constraints ###

# Each actor can only be in one movie
Const1a = m.addConstrs((ActorDec[:,i].sum() <= 1 for i in range(NumActors)))
Const1b = m.addConstrs((ActorDec[:,i].sum() >= 1 for i in range(NumActors)))
Const1c = m.addConstrs((ActorDec[i,:].sum() <= 2 for i in range(NumMovies)))
Const1d = m.addConstr((ActorDec[0:3, 0].sum()) == 0)
Const1e = m.addConstr((ActorDec[1:3, 2].sum()) == 0)
Const1f = m.addConstr((ActorDec[0:11, 6].sum()) >= 1)
Const1g = m.addConstr((ActorDec[8, 7]) == 1)

# Each Movie has to get a certain budget
Const2a = m.addConstrs((Budget[i] <= MaxBudget) for i in range(NumMovies))
Const2b = m.addConstrs((Budget[i] >= MinBudget) for i in range(NumMovies))
Const2c = m.addConstr((Budget.sum() <= TotalBudget))

# Each movie must be in a range of runtimes
Const3a = m.addConstrs((Runtime[i] <= MaxRuntime) for i in range(NumMovies))
Const3b = m.addConstrs((Runtime[i] >= MinRuntime) for i in range(NumMovies))


In [ ]:
### Objective Function ###
Revenue = np.array([])

#for i in range(NumMovies):
    #rev = GenreWeights[MovieGenres[i]] + (Runtime[i] * 8.499e+05) + (Budget[i] * 2.329e+00) + ActorDec[i] @ ActorWeights + (ReleaseYear * 1.051e+05)
    #np.append(Revenue, rev)

m.setObjective(sum(GenreWeights[MovieGenres[i]] for i in range(NumMovies))+sum(Budget[i] * 2.322e+00 for i in range(NumMovies)) + sum(Runtime[i]*1.138e+06 for i in range(NumMovies)) + sum(ActorDec[i] @ ActorWeights for i in range(NumMovies)), GRB.MAXIMIZE)
# insert predictive model
# m.setObjective(sum(Revenue), GRB.MAXIMIZE)
m.optimize()

m.getAttr("ObjVal")

#import pandas as pd
row_names = ['Comedy', 'Action', 'Action', 'Drama', 'Family', 'Horror', 'Animation', 'Science Fiction', 'Music', 'History']
col_names = ['Maggie Smith', 'Tom Hanks', 'Whoopi Goldberg', 'Orlando Bloom', 'Willem Dafoe', 'Bruce Willis', 'Alan Rickman', 'Scarlett Johansson', 'Leonardo DiCaprio', 'Chris Evans']

#pd.DataFrame(MY.getAttr('x'))

#pd.DataFrame(Budget.getAttr('x'), index = row_names, columns=["Budget Allocation"])
pd.DataFrame(ActorDec.getAttr('x'), index = row_names, columns=col_names)
#pd.DataFrame(Runtime.getAttr('x'))

,Budget Allocation
Comedy,30000000.0
Action,30000000.0
Action,30000000.0
Drama,30000000.0
Family,30000000.0
Horror,30000000.0
Animation,70000000.0
Science Fiction,250000000.0
Music,250000000.0
History,250000000.0
